In [ ]:
!pip install datasets
!pip install tqdm

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('key')
os.environ["KAGGLE_USERNAME"] = userdata.get('username')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import scipy

In [ ]:
!kaggle competitions download -c brain-motor-imagery-classification

 99% 417M/420M [00:20<00:00, 22.5MB/s]
100% 420M/420M [00:20<00:00, 21.5MB/s]


In [ ]:
!unzip /content/brain-motor-imagery-classification.zip

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Expss4/chunk_train_seq")

Generating train split:   0%|          | 0/5556 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['chunk_seq', 'label'],
        num_rows: 5556
    })
})

In [ ]:
train_df = pd.DataFrame(dataset["train"])

In [ ]:
train_df

,chunk_seq,label
0,"[[-150295.46875, -139498.671875, -169968.9375,...",[120]
1,"[[-150625.8125, -139779.859375, -170294.90625,...",[110]
2,"[[-150388.71875, -139349.8125, -170495.453125,...",[150]
3,"[[-150941.515625, -139904.671875, -170844.6718...",[150]
4,"[[-151937.0625, -140848.453125, -172032.796875...",[120]
...,...,...
5551,"[[-132026.125, -174318.765625, -194952.109375,...",[150]
5552,"[[-132115.09375, -174295.6875, -194905.4375, -...",[120]
5553,"[[-132171.5, -174317.0625, -194937.796875, -91...",[110]
5554,"[[-132222.109375, -174390.203125, -194980.2656...",[150]


In [ ]:
def clip_chunk(chunk):
    fix_length = 1730 # replace with min length
    trim = len(chunk) - fix_length
    # print(trim, len(chunk))
    if trim %2 == 0:
      trim_left = trim_right = int(trim/2)
      return chunk[trim_left:-trim_right]
    if trim %2 != 0:
      trim_left = trim//2 + 1
      trim_right = trim//2
      assert trim_left + trim_right == trim, 'error @clip_chunk'
      return chunk[trim_left:-trim_right]

In [ ]:
#down sampling
def down_sampling(signal_target):
    #f = signal.resample(sig, 1750)
    signa = np.transpose(signal_target)
    magic_num = 2
    dx = len(signal_target)//magic_num
    final = np.array([signal.resample(sub_sig, dx) for sub_sig in signa])
    #print(final.shape)
    final = np.transpose(final)
    #print(final.shape)

    return final

In [ ]:
# band-pass
def band_pass(high_cut:int, low_cut:int, sample_rate:int, n_order:int, signal_target:np.ndarray):
    #do fft
    # fft_signal_target = np.fft.fft2(signal_target)
    # fft_signal_target = np.transpose(fft_signal_target)
    #fft_signal_target = np.fft.fft2(signal_target)
    # signal_target = np.transpose(signal_target)
    #print(fft_signal_target.shape)

    ans = []
    for i in range(8):
        sos = scipy.signal.butter(n_order, [low_cut, high_cut], 'bandpass', fs=1/sample_rate, output='sos')
        filtered_data = scipy.signal.sosfiltfilt(sos, signal_target[:, i])
        #iffft_signal_target = np.fft.ifft2(filtered_data)
        ans.append(filtered_data)

    #iffft_signal_target = np.transpose(iffft_signal_target)

    return np.transpose(np.array(ans))

In [ ]:
# power-method
def power_method(signal_target):
    all_pow = []
    signal_target = np.transpose(signal_target)
    for chan in range(8):
        #processed_sig = np.fft.ifft(np.abs(np.fft.fft(signal_target[chan]))**2)
        fft_sig = np.abs(np.fft.fft(signal_target[chan]))
        processed_sig = fft_sig *fft_sig
        #print(processed_sig.shape)
        #all_pow.append(np.fft.ifft(np.abs(np.fft.fft(signal_target[:, chan]))**2))
        all_pow.append(np.fft.ifft(processed_sig))
    all_pow = np.transpose(np.array(all_pow))

    return all_pow

In [ ]:
from tqdm import tqdm
arrs = []
labels = []

for i in tqdm(range(len(train_df))):
    pivot = np.array(train_df["chunk_seq"][i])
    p_label = train_df["label"][i][0]
    try:
        # clip_signal = clip_chunk(pivot)
        # down_signal = down_sampling(clip_signal)
        # banded_signal = band_pass(32, 13, 1/125, 5, down_signal)
        #print(banded_signal.shape)
        #pow_signal = power_method(banded_signal)

        clip_signal = clip_chunk(pivot)
        banded_signal = band_pass(32, 13, 1/125, 5, clip_signal)
        #ica_signal = ICA(banded_signal)
        down_signal = down_sampling(banded_signal)



        #for j in range(8):
            # path = "train_stft_pre/"+f"{i}_{j}.jpg"
            # f, t, Zxx = signal.stft(banded_signal[:, j], 125, nperseg=865)
            # plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud')
            # plt.savefig(path)
            # img = Image.open(path)
            # plt.close()
            # imgs.append(img)
        arrs.append(down_signal)
        labels.append(p_label)
    except:
        print(i, "i just idk how it error!")


In [ ]:
arrs[0].shape

(867,)

In [ ]:
train_df_pre = pd.DataFrame({"arr":arrs, "label":labels})
train_df_pre

,arr,label
0,"[[0.007122313174213517, 0.24801742147367567, 0...",120
1,"[[-0.5374690486643031, 0.181824058094265, -0.1...",110
2,"[[0.15866651864620235, 0.023429035088381536, -...",150
3,"[[0.023477216026210994, -0.13785722158462596, ...",150
4,"[[0.2621378148501446, -0.041997332386340194, -...",120
...,...,...
5551,"[[0.26744272494772103, 2.720901250953109, 0.37...",150
5552,"[[0.5898926196094305, -0.007576879455883314, -...",120
5553,"[[0.21995950522166854, 0.17007442040565102, -0...",110
5554,"[[0.01684884665020385, 0.0633754323884602, -0....",150


In [ ]:
train_df_pre["arr"][0].shape

(865,)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import Dataset
import datasets

In [ ]:
train_df_pre = Dataset.from_dict({"arr":arrs, "label":labels})

In [ ]:
train_df_pre.push_to_hub("Expss4/train_seq_updated", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/317 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Expss4/train_seq_updated/commit/3d80e3482e7a93479a5165da87aa43ad233f3154', commit_message='Upload dataset', commit_description='', oid='3d80e3482e7a93479a5165da87aa43ad233f3154', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("Expss4/train_seq_pre_gj")

Generating train split:   0%|          | 0/44448 [00:00<?, ? examples/s]

In [ ]:
trainn = pd.DataFrame(dataset["train"])

In [ ]:
np.array(trainn["arr"][0]).shape

(865,)

In [ ]:
import glob

In [ ]:
import pandas as pd
submit = pd.read_csv("/content/sample_submission.csv")
submit

,id,predict
0,a0dfb1f6-a76b-404f-a04a-b61e955ce1ef,110.0
1,51afd110-7556-4a7d-b6d2-68e46256f267,150.0
2,eb229854-1a14-4e1f-8e52-470edbec47c9,150.0
3,8fa74efe-f269-415e-a968-1a92e7c66fe6,NaN
4,7b209881-a1dd-4aa3-a06a-037908b07bfa,NaN
...,...,...
475,69e84db3-0019-4e38-90fc-968f45747a16,NaN
476,d18d7d80-78fa-402a-ba1c-e35d72382b18,NaN
477,0a532391-dd42-413f-ab15-798960de95d6,NaN
478,0c02ad4f-de15-48cb-ad31-0925e69bfe97,NaN


In [ ]:
all_sig = []
all_path = submit["id"]
all_id = []

for i in tqdm(range(len(all_path))):
    path = "/content/test/"+submit["id"][i]+".npy"
    sigg = np.load(path)
    idd = path.replace("/content/test/", "")
    idd = idd.replace(".npy", "")
    all_sig.append(sigg)
    all_id.append(submit["id"][i])
    #print(idd, sigg.shape)
    #break

100%|██████████| 480/480 [00:00<00:00, 6058.16it/s]


In [ ]:
len(all_sig), len(all_id)

(480, 480)

In [ ]:
len(all_sig[0])

1751

In [ ]:
all_sig[0].shape

(1750, 17)

In [ ]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("Expss4/chunk_test_seq")

Generating train split:   0%|          | 0/480 [00:00<?, ? examples/s]

In [ ]:
test_df = pd.DataFrame(dataset["train"])

In [ ]:
test_df

,chunk_seq,id
0,"[[-115842.9296875, -132882.921875, -141490.937...",aa3a9edd-635b-4661-b44d-70ef4587002c
1,"[[-88666.6015625, -67764.3203125, -101396.8125...",9e6162c8-1e09-432d-b675-181207d8bd40
2,"[[-188023.859375, -171844.6875, -171932.484375...",bfe421b0-3eee-40b5-97eb-1c227cc2b5f7
3,"[[-115382.9296875, -131498.8125, -139349.35937...",56677f18-9b7a-41e1-9844-81eb070b61b1
4,"[[-116413.8828125, -133430.171875, -142606.906...",81c7aa80-9cb1-418a-8618-9429afb23c3e
...,...,...
475,"[[-190029.359375, -173065.453125, -172679.2187...",8cfffbe1-4317-4eca-b17b-f2a43844a663
476,"[[-116050.171875, -133123.953125, -141980.5156...",b075aa5c-a564-4f01-b443-9410e1e4277e
477,"[[-105921.2578125, -107282.03125, -120265.1796...",e793e44c-4620-4fee-b709-94026c4dad02
478,"[[-194069.203125, -174909.296875, -173640.25, ...",0e205244-a95f-45c8-b9e0-031f382fb104


In [ ]:
sigs = []
labels = []

for i in tqdm(range(480)):
    sample_signal = np.array(all_sig[i])
    clip_signal = clip_chunk(sample_signal)
    banded_signal = band_pass(32, 13, 1/125, 5, clip_signal)
    ica_signal = ICA(banded_signal)
    down_signal = down_sampling(ica_signal.T)

    # for j in range(8):
    sigs.append(down_signal)
    labels.append(submit["id"][i])
    # if(i==1):
    #     break

In [ ]:
len(sigs), len(labels)

(480, 480)

In [ ]:
labels[0]

'480b6f84-33fe-4ca2-ac38-aa42f3bfa6b4'

In [ ]:
sigs[0].shape

(865,)

In [ ]:
test_df_pre = Dataset.from_dict({"arr":sigs, "id":labels})

In [ ]:
test_df_pre.push_to_hub("Expss4/test_seq_updated", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Expss4/test_seq_updated/commit/fac9ccad2dea93a23de2f3c16049b028aa7e8e97', commit_message='Upload dataset', commit_description='', oid='fac9ccad2dea93a23de2f3c16049b028aa7e8e97', pr_url=None, pr_revision=None, pr_num=None)